<a href="https://colab.research.google.com/github/ketanhdoshi/ml/blob/master/lib/training_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Neural Net Training Loop


**Todos**
*   DebugYhatCB should be renamed, and stop printing things to screen - so change all usages of it in arch_lib etc
*   DebugTracker should be auto created in Trainer but turned off with max_count = 0. Then you can call dtr.set_settings() and pass in all the disp, freq etc settings to activate TB etc.
*   Clean up repr. Aand tb_results is still a bit messy
*   Take photos of drawings in my paper notebooks
*   Trainer should take a debug=True boolean, and create and initialise DTR within itself
*   !!!!! IMP - DTR add_param and add_activation should be wrapped in torch.no_grad
*   dtr.pd_results() should return separate run_df and epoch_df
*   Get rid of MetricsCB and remove references to it from all files.
*   Add dtr.compare_results() which calls dtr.pd.compare_batch_results() etc.
*   Enhance the training loop so that it can pass multiple 'x' arguments to the model. So if the 'x' value is a list of values, it destructures them as *xb and passes them in as multiple arguments. Then use that with a Tuple Dataset (see Todos in data_lib) or with Tabular ItemList, pass the continuous and categorical as separate arguments
*   DONE Enhance the training loop so that it can pass multiple 'y' arguments to the model. So if the 'y' value is a tuple of values, it destructures them as *yb and passes them in as multiple arguments.

**Done**
1. DONE Remove torch.no_grad from Callback processing
2. DONE Add torch.no_grad in MetricsCB calc metrics
. DONE Run Fastai callbacks notebook to see the effect of adding/removing torch.no_grad in AvgStats
5. DONE Add DebugTrackRun graphical plotting
7. DONE Tensorboard Pytorch
9. DONE Document GANs
10. DONE Document L1 and L2 regularisation
12. DONE Complete ULMFit with datalib
13. DONE Use datalib for CycleGAN and DONE - CUDA
4. DONE DebugTrackRun - add fine-grained control to track only selected steps (just like we can track selected batches only)
15. DONE (via Pandas) Comparison between two steps for the same data parameter
16. DONE Clean up get/show results, activation/param_steps, debugtrackrun should be avbl in ctx, set_display, rename variables for operation, step etc
17. DONE (via Pandas) DebugTrackRun - results with 3 separate lists for run, batch and steps
18. DONE Output to Print Display (can be done via Pandas) and Dataframe ie. create DebugPrint and DebugDataframe
23. DONE Add Comments to data_lib (GAN bundle), training_lib (Debug) and Cycle GAN
24. DONE Move this file into lib and change the export
24. DONE Recorder integrated with DebugTracker?
24. DONE Use the timestr as the run_idx
24. DONE DebugTracker output to SNS/Matplotlib? Maybe do that via pandas
24. DONE Add DebugTrackRun comparison between two runs
24. DONE Improve Hooks debugging capabilities and log to DebugTracker
24. DONE Metrics and Hyperparams tracking going to DebugTracker. Rationalise with trainer.logger which Metrics and Progress use for output.
24. DONE fix the accumulate_batch logic. Problem is that for Transformers the shape of yhat is (smaples, tgt_vocab_sz, seq) and since the seq len is ifferent in each batch, the yhat dimensions are different from one batch to the next and cannot be concatenated during accumulation







### Import Libraries

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
#export

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial
import time
import pickle
from collections import OrderedDict
from pathlib import *

from fastprogress import master_bar, progress_bar
from fastprogress.fastprogress import format_time

import torch
from torch import tensor
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#!pip install torchviz
import IPython.core.debugger as db

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
gn_path = 'gdrive/My Drive/Colab Notebooks'  #change dir to your project folder

import sys
sys.path.insert(1, gn_path + '/exp')

from nb_util import get_mnist_data, accuracy
from nb_data import MultiDimDataset, get_dls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
#export

from nb_data import Plotter

### Enhanced Training Loop (Step 6 - Create a Trainer class and Include Callbacks)

In [ ]:
#export

#----------------------------------------------------
# Define the list of callback events
#----------------------------------------------------
EV_BEGIN_FIT='begin_fit'
EV_BEGIN_EPOCH='begin_epoch'

EV_BEGIN_TR='begin_tr'
EV_BEGIN_TR_BATCH='begin_tr_batch'
EV_AFTER_TR_PRED='after_tr_pred'
EV_AFTER_TR_LOSS='after_tr_loss'
EV_AFTER_TR_BACKWARD='after_tr_backward'
EV_AFTER_TR_OPTSTEP='after_tr_optstep'
EV_END_TR_BATCH='end_tr_batch'
EV_END_TR='end_tr'
EV_CANCEL_TR_BATCH='cancel_tr_batch'
EV_CANCEL_TR='cancel_tr'

EV_BEGIN_VAL='begin_val'
EV_BEGIN_VAL_BATCH='begin_val_batch'
EV_AFTER_VAL_PRED='after_val_pred'
EV_AFTER_VAL_LOSS='after_val_loss'
EV_END_VAL_BATCH='end_val_batch'
EV_END_VAL='end_val'
EV_CANCEL_VAL_BATCH='cancel_val_batch'
EV_CANCEL_VAL='cancel_val'

EV_END_EPOCH='end_epoch'
EV_END_FIT='end_fit'
EV_CANCEL_FIT='cancel_fit'

#----------------------------------------------------
# Define Exceptions for aborting the callback chain
#----------------------------------------------------
class CancelFitException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

#----------------------------------------------------
# Base Callback class
#----------------------------------------------------
class Callback():
  def process(self, event, ctx):
    f = getattr(self, event, None)
    if f:
      f(ctx)

#----------------------------------------------------
# Class that runs the training and validation loop
# It keeps track of the following while the loop is being executed:
#     - Training and Validation DataLoader
#     - Model, Optimiser, Loss Function, Callbacks, Hooks and Tracker
#     - Number of Epochs (num_epochs) and Number of Batches (num_batches) in Current Epoch
#     - Current Epoch (i_epoch) and Current Batch (i_batch) within Current Epoch
#     - Current Training Batch Iteration (i_tr_iter) across all Batches so far
#     - Data (xb) and Target Labels (yb) in Current Batch
#     - Predicted Value (yhat) and Loss Value (loss)
#----------------------------------------------------
class Trainer():
  def __init__(self, tr_dl, val_dl, model, opt, loss_func, cbs=None, dtr=None):
    self.tr_dl, self.val_dl = tr_dl, val_dl
    self.model, self.opt, self.loss_func = model, opt, loss_func
    self.cbs = cbs
    self.dtr = dtr
    if (dtr is not None):
      assert(dtr in cbs)
    self.logger = print
    
  def add_cb (self, cb):
    self.cbs.append(cb)
    
  #----------------------------------------------------
  # Fire a callback event
  #----------------------------------------------------
  def fire(self, event, **kwargs):
    for key, value in kwargs.items(): 
      #print ("%s == %s" %(key, value))
      setattr(self, key, value)
    for cb in self.cbs:
      cb.process(event, self)
    
  #----------------------------------------------------
  # Run one training batch
  #----------------------------------------------------
  def _tr_batch(self, i_batch, xb, yb):
    try:
      self.fire(EV_BEGIN_TR_BATCH, i_batch=i_batch, xb=xb, yb=yb, i_tr_iter=self.i_tr_iter+1)

      # Run the forward pass and compute the loss
      self.yhat = self.model(self.xb)
      self.fire(EV_AFTER_TR_PRED)

      yb = self.yb if isinstance(self.yb, list) else [self.yb]
      self.loss = self.loss_func(self.yhat, *yb)
      self.fire(EV_AFTER_TR_LOSS)

      # Run the backward pass to compute gradients
      self.loss.backward()
      self.fire(EV_AFTER_TR_BACKWARD)
        
      # Now optimiser uses the gradients to update the weights and biases of all layers
      self.opt.step()
      self.fire(EV_AFTER_TR_OPTSTEP)

      self.opt.zero_grad()
      
    except CancelBatchException:
      self.fire(EV_CANCEL_TR_BATCH)
    finally:
      self.fire(EV_END_TR_BATCH)
    
  #----------------------------------------------------
  # Run one epoch of training batches
  #----------------------------------------------------
  def _tr_epoch(self):
    try:
      self.fire(EV_BEGIN_TR, num_batches=len(self.tr_dl))
      self.model.train()
      
      # Get data for the next batch
      for i_batch, (xb, yb) in enumerate(self.tr_dl):
        self._tr_batch(i_batch, xb, yb)

    except CancelEpochException:
      self.fire(EV_CANCEL_TR)
    finally:
      self.fire(EV_END_TR)    

  #----------------------------------------------------
  # Run one validation batch
  #----------------------------------------------------
  def _val_batch(self, i_batch, xb, yb):
    try:
      self.fire(EV_BEGIN_VAL_BATCH, i_batch=i_batch, xb=xb, yb=yb)

      self.yhat = self.model(self.xb)
      self.fire(EV_AFTER_VAL_PRED)

      yb = self.yb if isinstance(self.yb, list) else [self.yb]
      self.loss = self.loss_func(self.yhat, *yb)
      self.fire(EV_AFTER_VAL_LOSS)
            
      self.fire(EV_END_VAL_BATCH)
      
    except CancelBatchException:
      self.fire(EV_CANCEL_VAL_BATCH)
    finally:
      self.fire(EV_END_VAL_BATCH)
    
  #----------------------------------------------------
  # Run one epoch of validation batches
  #----------------------------------------------------
  def _val_epoch(self):
    try:
      self.fire(EV_BEGIN_VAL, num_batches=len(self.val_dl))
      # Run the validation loop
      self.model.eval()

      with torch.no_grad():
        
        # Get data for the next batch
        for i_batch, (xb, yb) in enumerate(self.val_dl):
          self._val_batch(i_batch, xb, yb)

    except CancelEpochException:
      self.fire(EV_CANCEL_VAL)
    finally:
      self.fire(EV_END_VAL)
    
  #----------------------------------------------------
  # Run the complete loop for all epochs
  #----------------------------------------------------
  def fit(self, num_epochs):
    try:
      self.fire(EV_BEGIN_FIT, num_epochs=num_epochs, i_tr_iter=-1)

      for i_epoch in range(self.num_epochs):
        self.fire(EV_BEGIN_EPOCH, i_epoch=i_epoch)

        self._tr_epoch()
        if (self.val_dl):
          self._val_epoch()

        self.fire(EV_END_EPOCH)

    except CancelFitException:
      self.fire(EV_CANCEL_FIT)
    finally:
      self.fire(EV_END_FIT)

### Callbacks - Metrics, Abort, Progress

In [ ]:
#export

class MetricsGrp(Callback):
  def __init__(self, metrics):
    self.metrics = metrics
    # !!!!! Register callbacks for each metric

  # ----------------------------
  # Print headers for all the result columns
  # ----------------------------   
  def begin_fit(self, ctx):
    # ['epoch', 'tr_loss', 'smooth_loss', 'val_loss', 'metric1', 'metric2', ..'time']
    headers = ['epoch']
    for metric in self.metrics: headers += metric.header
    headers += ['time']
    ctx.logger(headers)

  # ----------------------------
  # Track start time at the beginning of each epoch
  # ----------------------------   
  def begin_epoch(self, ctx):
    ctx.start_time = time.time()

  # ----------------------------
  # Print results at the end of each epoch
  # ----------------------------   
  def end_epoch(self, ctx):
    # Flattened metric results and epoch time taken
    values = [results for metric in self.metrics for results in metric._results()]
    epoch_time = [format_time(time.time() - ctx.start_time)]
    #for metric in self.metrics: values += metric._results()

    # Epoch number, formatted metric results, and epoch time
    res = [str(ctx.i_epoch)]
    res += [f'{val:.6f}' for val in values]
    res += epoch_time

    ctx.logger(res)

    # Report metrics to the DTR
    if (ctx.dtr): self._report(ctx.dtr, values + epoch_time)

  # ----------------------------
  # Report metrics to the DTR
  # ----------------------------   
  def _report(self, dtr, results):
    headers = [header for metric in self.metrics for header in metric.header] + ['time']
    epoch_metrics = {header:result for header, result in zip(headers, results)}
    dtr.update_epoch(epoch_metrics)

#----------------------------------------------------
# Base class for all Metrics callbacks
#
# It handles some common functionality such as managing the metric result value (one or more)
# that it computes per epoch. It resets that value at the
# end of each epoch. It provides abstract methods to do any necessary computations
# at the end of each batch and at the end of each epoch. The child class provides
# implementations for these methods. It also accumulates the inputs (ie. preds and targs)
# after each batch, which the child class can use for its end-epoch calculations
# if it needs.
#----------------------------------------------------
class MetricsBaseCB(Callback):
  header = ['metric']

  def __init__(self, accumulate=False):
    self.num_value = len(self.header)
    self.accumulate = accumulate

  # ----------------------------
  # Reset any state, when training starts and at beginning of each epoch
  # ----------------------------   
  def _reset(self, begin_fit=False):
    pass

  # ----------------------------
  # Return the metric values
  # ----------------------------   
  def _results(self):
    values = [self.value] if (self.num_value == 1) else self.value
    values = [val.item() if (isinstance(val, torch.Tensor)) else val for val in values]
    return values

  # ----------------------------
  # Do any required calculations at the end of the batch. A flag indicates if it
  # is a training or validation batch.
  # Functionality gets implemented within the child callback.
  # ----------------------------   
  def _calc_batch(self, ctx, is_val=True):
    pass

  # ----------------------------
  # Reset the accumulated values
  # ----------------------------   
  def _reset_accumulate(self):
    # Accumulated 'yhat' predictions and 'yb' targets
    self.yhat, self.yb = None, None

  # ----------------------------
  # Accumulate the target and preds after each batch so that they
  # can be used for end-of-epoch calculations
  # ----------------------------   
  def _accumulate_batch(self, ctx):
    if (not self.accumulate):
      return

    if (self.yhat is None):
      # Initialise for the first batch of the epoch. Cloning copies the type, device
      # and size of the tensor.
      self.yhat = ctx.yhat.clone().detach()
      self.yb = ctx.yb.clone().detach()
    else:
      # Concatenate this batch data to the accumulated data from previous batches
      self.yhat = torch.cat((self.yhat, ctx.yhat.detach()))
      self.yb = torch.cat((self.yb, ctx.yb.detach()))

  # ----------------------------
  # Do any required calculations at the end of the epoch
  # ----------------------------   
  def _calc_epoch(self, ctx):
    pass

  # ----------------------------
  # Reset all metrics when training starts
  # ----------------------------   
  def begin_fit(self, ctx):
    self._reset(begin_fit=True)

  # ----------------------------
  # Reset all metrics at the beginning of each epoch
  # ----------------------------   
  def begin_epoch(self, ctx):
    self.value = 0. if (self.num_value == 1) else [0.] * self.num_value
    self._reset_accumulate()
    self._reset()

  # ----------------------------
  # Calculate training metrics for each batch
  # ----------------------------   
  def after_tr_loss(self, ctx):
    with torch.no_grad():
      self._calc_batch(ctx, is_val=False)

  # ----------------------------
  # Calculate validation metrics for each batch
  # ----------------------------   
  def after_val_loss(self, ctx):
    self._calc_batch(ctx)
    self._accumulate_batch(ctx)

  # ----------------------------
  # Print results at the end of each epoch
  # ----------------------------   
  def end_epoch(self, ctx):
    with torch.no_grad():
      self._calc_epoch(ctx)

#----------------------------------------------------
# Calculate an Average value per epoch using the given metric function
#----------------------------------------------------
class AverageMetricsCB(MetricsBaseCB):
  header = ['avg']

  def __init__(self, metric_fn):
    self.metric_fn = metric_fn
    self.header = [metric_fn.__name__]
    super().__init__()

  # ----------------------------
  # Count of total number of sample rows in an epoch
  # ----------------------------
  def _reset(self, begin_fit=False):
    self.total_sz = 0

  # ----------------------------
  # Add up the value returned by the metric function for each batch. It will be
  # averaged at the end of the epoch. Metrics are computed only for validation batches.
  # ----------------------------   
  def _calc_batch(self, ctx, is_val=True):
    if (is_val):
      batch_sz = ctx.yb.size(0)
      self.total_sz += batch_sz
      self.value += self.metric_fn(ctx.yhat, ctx.yb) * batch_sz

  # ----------------------------
  # Compute the average at the end of the epoch
  # ----------------------------
  def _calc_epoch(self, ctx):
    self.value = self.value / self.total_sz

#----------------------------------------------------
# Track the Training and Validation Loss per epoch, already 
# calculated by the loss function.
# This is the only metric which is computed for both training and validation
#----------------------------------------------------
class LossMetricsCB(MetricsBaseCB):
  header = ['tr_loss', 'smooth_loss', 'val_loss']
  beta=0.98

  # ----------------------------
  # Count of total number of sample rows in an epoch
  # ----------------------------
  def _reset(self, begin_fit=False):
    self.total_sz = 0

    # Initialise only once when training starts, but don't reset after each epoch
    if (begin_fit):
      self.mov_avg = 0
      self.n = 0

  # ----------------------------
  # Add up the loss for each training and validation batch
  # ----------------------------   
  def _calc_batch(self, ctx, is_val=True):
    if (is_val):
      # Validation loss
      batch_sz = ctx.yb.size(0)
      self.total_sz += batch_sz
      self.value[2] += ctx.loss * batch_sz
    else:
      # Training loss
      loss = ctx.loss
      self.value[0] += ctx.loss

      # Smoothed training loss based on moving average
      self.n += 1
      self.mov_avg = self.beta * self.mov_avg + (1 - self.beta) * loss
      self.value[1] = self.mov_avg / (1 - self.beta ** self.n)

  # ----------------------------
  # Compute the average at the end of the epoch
  # ----------------------------
  def _calc_epoch(self, ctx):
    # Calculate Training loss
    # Smoothed Training loss is already updated after each batch
    self.value[0] = self.value[0] / len(ctx.tr_dl)

    # Calculate Validation loss
    if (self.total_sz > 0):
      self.value[2] = self.value[2] / self.total_sz

#----------------------------------------------------
# Roc Auc score per epoch
#----------------------------------------------------
from sklearn.metrics import roc_auc_score
class RocAucMetricsCB(MetricsBaseCB):
  header = ['roc_auc']

  def __init__(self):
    super().__init__(accumulate=True)

  def _calc_epoch(self, ctx):
    yhat = torch.sigmoid(self.yhat)
    #yhat = F.softmax(ctx.yhat, dim=1)[:,-1]
    roc_score = roc_auc_score(self.yb.cpu().numpy(), yhat.cpu().numpy())
    self.value = roc_score


In [ ]:
#export

#----------------------------------------------------
# Callback to calculate training and validation metrics
# We are given a dictionary of metric names and metric functions {'accuracy': accuracy_fn, 'f1': f1_fn, ...}
#----------------------------------------------------
class MetricsCB(Callback):
  def __init__(self, metric_dict):
    self.metric_dict = metric_dict
    self._reset()
  
  # ----------------------------
  # Reset all our metrics to 0
  # ----------------------------   
  def _reset(self):
    # Reset training and validation metrics
    # Training metrics are [metric1_tr_value, metric2_tr_value, ...]
    # Similarly, validation metrics are [metric1_val_value, metric2_val_value, ...]
    self.metrics_tr = len(self.metric_dict) * [0.]
    self.metrics_val = len(self.metric_dict) * [0.]

    # Reset total training loss and total validation loss
    self.tot_loss_tr = 0.
    self.tot_loss_val = 0.
  
  # ----------------------------
  # Format the metric values for training or validation. Calculate the average
  # value of that metric by dividing by number of iterations
  # ----------------------------   
  def _results(self, tot_loss, metrics, nv):
    # Format the loss value
    res = ['{0:.6f}'.format(tot_loss/nv)]

    # Format the metric values
    for i, metric_name in enumerate(self.metric_dict.keys()):
      res += ['{0:.6f}'.format(metrics[i]/nv)]

    return (res)

  # ----------------------------
  # Calculate values of each metric using the metric function and accumulate it
  # by adding it to the sum of all previous values of that metric 
  # ----------------------------   
  def _calc_metrics(self, metrics, ctx):
    for i, metric_fn in enumerate(self.metric_dict.values()):
      metrics[i] += metric_fn(ctx.yhat, ctx.yb)

  # ----------------------------
  # Print headers for all the result columns
  # ----------------------------   
  def begin_fit(self, ctx):
    # ['loss', 'metric1', 'metric2', ..]
    metric_names = ['loss'] + list(self.metric_dict.keys())
    # ['epoch', ...metric names..., 'time']
    headers = ['epoch'] + [f'train_{n}' for n in metric_names] + [f'valid_{n}' for n in metric_names] + ['time']
    ctx.logger(headers)

  # ----------------------------
  # Reset all metrics at the beginning of each epoch
  # ----------------------------   
  def begin_epoch(self, ctx):
    self._reset()
    ctx.start_time = time.time()

  # ----------------------------
  # Calculate training metrics for each batch
  # ----------------------------   
  def after_tr_loss(self, ctx):
    with torch.no_grad():
      self.tot_loss_tr += ctx.loss
      self._calc_metrics(self.metrics_tr, ctx)

  # ----------------------------
  # Calculate validation metrics for each batch
  # ----------------------------   
  def after_val_loss(self, ctx):
    self.tot_loss_val += ctx.loss
    self._calc_metrics(self.metrics_val, ctx)

  # ----------------------------
  # Print results at the end of each epoch
  # ----------------------------   
  def end_epoch(self, ctx):  
    res = [str(ctx.i_epoch)]
    res += self._results(self.tot_loss_tr, self.metrics_tr, len(ctx.tr_dl))
    res += self._results(self.tot_loss_val, self.metrics_val, len(ctx.val_dl))
    res += [format_time(time.time() - ctx.start_time)]
    ctx.logger(res)
  

In [ ]:
#export

#----------------------------------------------------
# Callback to test aborting of training
#----------------------------------------------------
class AbortTrainCB(Callback):
  def __init__(self, abort_iter=5):
    self.abort_iter = abort_iter

  #def begin_tr_batch(self, ctx):
  #  print(ctx.num_epochs, ctx.i_epoch, ctx.num_batches, ctx.i_batch, ctx.i_tr_iter)

  def after_tr_optstep(self, ctx):
    # +1 because i_tr_iter starts from 0
    if (ctx.i_tr_iter + 1 >= self.abort_iter):
      raise CancelFitException()
  
#----------------------------------------------------
# Callback to show progress indicators using Fastai's progress widget
# Two progress bars are displayed - the first one shows the number of epochs and
# the second one shows the batches within the epoch
#----------------------------------------------------
class ProgressCallback(Callback):
  def begin_fit(self, ctx):
    # master_bar displays the epoch progress and the child progress_bar displays the batch progress. 
    # We create one at the beginning of each training/validation phase, and update it at the end of each batch. 
    self.mbar = master_bar(range(ctx.num_epochs))
    self.mbar.on_iter_begin()
    # Change the logger to the write function of the master bar, so everything gets written to the progress bar
    ctx.logger = partial(self.mbar.write, table=True)
        
  def end_fit(self, ctx): self.mbar.on_iter_end()
  def end_tr_batch(self, ctx): self.pbar.update(ctx.i_batch)
  def end_val_batch(self, ctx): self.pbar.update(ctx.i_batch)
  def begin_tr(self, ctx): self.set_pbar(ctx.tr_dl, ctx.i_epoch)
  def begin_val(self, ctx): self.set_pbar(ctx.val_dl, ctx.i_epoch)
        
  def set_pbar(self, dl, i_epoch):
    self.pbar = progress_bar(dl, parent=self.mbar)
    self.mbar.update(i_epoch)

In [ ]:
def get_linear_model(inp_size, n_classes, lr=0.5, nh=50):
    model = nn.Sequential(nn.Linear(inp_size,nh), nn.ReLU(), nn.Linear(nh, n_classes))
    return model, optim.SGD(model.parameters(), lr=lr)
  
def run_mnist(cbs, num_epochs, repro=False, dtr=None):
  x_train,y_train,x_valid,y_valid = get_mnist_data()
  train_ds,valid_ds = MultiDimDataset(x_train, y_train), MultiDimDataset(x_valid, y_valid)
  train_dl,valid_dl = get_dls(train_ds, valid_ds, bs=512)
  n_classes = y_train.max().item()+1
  loss_func = F.cross_entropy

  if (repro):
    torch.manual_seed(0)
  model, opt = get_linear_model(train_dl.dataset.x.shape[1], n_classes=n_classes, lr=0.5, nh=50)
  loop = Trainer(train_dl, valid_dl, model, opt, loss_func, cbs, dtr)
  loop.fit(num_epochs)
  return(loop)
  
abort_cbs = [AbortTrainCB()]
metrics_cbs = [MetricsCB({"acc": accuracy})]
progress_cbs = [ProgressCallback()] + metrics_cbs
_ = run_mnist(metrics_cbs, 1, True)
_ = run_mnist(progress_cbs, 2, True)
_ = run_mnist(abort_cbs, 1, True)

['epoch', 'train_loss', 'train_acc', 'valid_loss', 'valid_acc', 'time']
['0', '0.675446', '0.800317', '0.303038', '0.912171', '00:00']


epoch,train_loss,train_acc,valid_loss,valid_acc,time
0,0.675446,0.800317,0.303038,0.912171,00:00
1,0.297040,0.912553,0.248718,0.929237,00:00


### Cuda Callback

In [ ]:
#export

#----------------------------------------------------
# Callback which puts the model, inputs and targets on the same device
#----------------------------------------------------
class CudaCB(Callback):
  def __init__(self, device):
    self.device=device

  def _do_data(self, data):
    if (isinstance(data, list)):
      device_data = [d.to(self.device) for d in data]
    else:
      device_data = data.to(self.device)
    return device_data

  def _do_batch(self, ctx):
    #ctx.xb, ctx.yb = ctx.xb.to(self.device), ctx.yb.to(self.device)
    ctx.xb, ctx.yb = self._do_data (ctx.xb), self._do_data (ctx.yb)
  
  def begin_fit(self, ctx):
    ctx.model.to(self.device)
    
  def begin_tr_batch(self, ctx):
    self._do_batch(ctx)

  def begin_val_batch(self, ctx): 
    self._do_batch(ctx)

### Tensorboard and Pandas Integration

In [ ]:
#export
#----------------------------------------------------
# Pytorch integrates with Tensorboard via torch.utils.tensorboard.SummaryWriter, which is 
# a Pytorch-compatible version of Tensorflow's SummaryWriter, and is used in a similar way
# 
# The SummaryWriter must be opened, giving it a file path where it writes the metrics.
# Then any results that you want to track can be written to this SummaryWriter. Finally
# that data can be flushed and the SummaryWriter is closed.
#----------------------------------------------------


#----------------------------------------------------
# Outputs debug results to Tensorboard for visualisation
#----------------------------------------------------
class DebugTensorboard():
  def __init__(self, log_dir='tbtry'):
    self.run_id = None
    self.writer = None
    self.log_dir = Path.cwd() / log_dir

  #----------------------------------------------------
  # Initialise the Summary Writer at the start of a run, so that result data can be written to it
  # for visualisation by Tensorboard
  #----------------------------------------------------
  def start_run(self, run_id):
    self.run_id = run_id
    run_path = self.log_dir / self.run_id 
    self.writer = SummaryWriter(run_path)

  #----------------------------------------------------
  # Close the Summary Writer at the end of a run, so that result data written so far is flushed
  # to Tensorboard
  #----------------------------------------------------
  def end_run(self):
    self.writer.close()

  #----------------------------------------------------
  # Histogram
  #----------------------------------------------------
  def add_hist(self, tag, data, step):
    self.writer.add_histogram(tag, data, step)

  #----------------------------------------------------
  # Scalars
  #----------------------------------------------------
  def add_scalar(self, tag, data, step):
    self.writer.add_scalar(tag, data, step)

  #----------------------------------------------------
  # Image and Model Graph
  #----------------------------------------------------
  def add_image_graph(self, arch, images):
    grid = torchvision.utils.make_grid(images)
    writer.add_image('images', grid, 0)
    writer.add_graph(arch, images)

In [ ]:
#export

#----------------------------------------------------
# Outputs debug results to Pandas
#----------------------------------------------------
class DebugPandas():
  #----------------------------------------------------
  # Create a tuple of 4 dataframes from the result rows
  # (run + epoch rows, batch rows, layer rows, all rows)
  #----------------------------------------------------
  def create_dfs (self, all_rows):
    all_df = pd.DataFrame(all_rows)
    run_df = self._create_subdf(all_df, 'run')
    epoch_df = self._create_subdf(all_df, 'epoch')
    run_df = run_df.append(epoch_df)

    batch_df = self._create_subdf(all_df, 'batch')
    layer_df = self._create_subdf(all_df, 'layer')
    return ((run_df, batch_df, layer_df, all_df))

  #----------------------------------------------------
  # Create a subset of the full dataframe containing only the rows
  # of the given row 'sub_type'
  #----------------------------------------------------
  def _create_subdf(self, all_df, sub_type):
    # Select only the rows of the given sub type
    sub_df = all_df[all_df['row_type'] == sub_type]

    # Exclude any columns which have only NaNs everywhere. 
    sub_cols = sub_df.columns[sub_df.notna().any()].tolist()
    sub_df = sub_df[sub_cols]

    return sub_df

  #----------------------------------------------------
  # Plot standard metrics after a training run, based on the result dataframes.
  # Columns that contain results for any additional application-specific metrics can 
  # also be passed in.
  #----------------------------------------------------
  @staticmethod
  def display_metrics(run_df, batch_df, app_metrics=[]):
    p = Plotter()
    run_cols = ['tr_loss', 'smooth_loss', 'val_loss'] + app_metrics
    run_df[run_cols] = run_df[run_cols].astype('float64')
    p.plot_grid ('line', run_df, run_cols, 'epoch_id')
    p.plot_grid ('line', batch_df, ['loss', 'lr', 'mom'], 'iter', hue="batch_type")
    p.create_subplot ('line', batch_df[batch_df['batch_type'] == 'train'], 'iter', 'loss')
    p.create_subplot ('line', batch_df[batch_df['batch_type'] == 'val'], 'iter', 'loss')

  #----------------------------------------------------
  # TODO: !!!!!!!!! This is the method that should be called externally to compare
  # the results of two different runs. Internally, this method should call the below
  # methods to compare batch_results, activation_results and param_results and then
  # return a consolidated status.
  #----------------------------------------------------
  def compare_results(dtr1, dtr2):
    pass

  #----------------------------------------------------
  # Compare the batch result dataframes for two runs. If they are equal it prints a
  # status of 'Matched'. It returns a merged dataframe with the differences of the
  # important columns.
  #----------------------------------------------------
  @staticmethod
  def compare_batch_results(batch_df1, batch_df2):
    batch_df1 = batch_df1[['batch_type', 'batch_id', 'pred', 'loss', 'lr']].reset_index(drop=True)
    len_df1 = len(batch_df1)
    batch_df2 = batch_df2[['batch_type', 'batch_id', 'pred', 'loss', 'lr']].reset_index(drop=True)
    batch_df2 = batch_df2[:len_df1].reset_index(drop=True)

    diff_df = batch_df1.merge(batch_df2, how='inner', left_on=['batch_type', 'batch_id'], right_on=['batch_type', 'batch_id'], suffixes=('_1', '_2'))
    assert(len(diff_df) == len_df1)
    diff_df['diff_pred'] = diff_df['pred_1'] - diff_df['pred_2']
    diff_df['diff_loss'] = diff_df['loss_1'] - diff_df['loss_2']
    diff_df['diff_lr'] = diff_df['lr_1'] - diff_df['lr_2']

    unmatched_df = diff_df[(diff_df['diff_loss'] != 0.0)]
    if (len(unmatched_df) != 0):
      print ('Unmatched')
    else:
      print ('Matched')
    return diff_df

  #----------------------------------------------------
  # Compare the layer activation result dataframes for two runs. If they are equal it prints a
  # status of 'Matched'. It returns a merged dataframe with the differences of the
  # important columns.
  #----------------------------------------------------
  @staticmethod
  def compare_activation_results(layer_df1, layer_df2):
    # Get the activation rows
    layer_df1 = layer_df1[(layer_df1['step_type'] == 'activation')]
    layer_df2 = layer_df2[(layer_df2['step_type'] == 'activation')]

    # Nothing to do if there are no activation rows
    len_df1 = len(layer_df1)
    len_df2 = len(layer_df2)
    if ((len_df1 == 0) or (len_df2 == 0)):
      print (f'No activation rows. df1: {len_df1}, df2: {len_df2}')
      return None

    # Get the columns to be matched
    layer_df1 = layer_df1[['batch_type', 'batch_id', 'layer_name', 'out_activation']].reset_index(drop=True)
    layer_df2 = layer_df2[['batch_type', 'batch_id', 'layer_name', 'out_activation']].reset_index(drop=True)

    # If one dataframe has more rows, get the same number of rows from both dataframes
    layer_df2 = layer_df2[:len_df1].reset_index(drop=True)

    # Merge the columns of the two dataframes by joining on batch id
    diff_df = layer_df1.merge(layer_df2, how='inner', left_on=['batch_type', 'batch_id', 'layer_name'], right_on=['batch_type', 'batch_id', 'layer_name'], suffixes=('_1', '_2'))

    # Check that all rows got merged
    assert(len(diff_df) == len_df1)

    # Check that there is no difference between the activation values of both dataframes
    diff_df['diff_activation'] = diff_df['out_activation_1'] - diff_df['out_activation_2']
    unmatched_df = diff_df[(diff_df['diff_activation'] != 0.0)]

    if (len(unmatched_df) != 0):
      print ('Unmatched')
    else:
      print ('Matched')

    return diff_df

  #----------------------------------------------------
  # Compare the layer param result dataframes for two runs. If they are equal it prints a
  # status of 'Matched'. It returns a merged dataframe with the differences of the
  # important columns.
  #----------------------------------------------------
  @staticmethod
  def compare_param_results(layer_df1, layer_df2):
    # Get the param rows
    layer_df1 = layer_df1[(layer_df1['step_type'] == 'param') & (layer_df1['batch_type'] == 'train')]
    layer_df2 = layer_df2[(layer_df2['step_type'] == 'param') & (layer_df2['batch_type'] == 'train')]

    # Nothing to do if there are no param rows
    len_df1 = len(layer_df1)
    len_df2 = len(layer_df2)
    if ((len_df1 == 0) or (len_df2 == 0)):
      print (f'No param rows. df1: {len_df1}, df2: {len_df2}')
      return None
    
    # Get the columns to be matched
    layer_df1 = layer_df1[['batch_type', 'batch_id', 'layer_name', 'param_mean', 'param_std', 'grad_mean', 'grad_std']].reset_index(drop=True)
    layer_df2 = layer_df2[['batch_type', 'batch_id', 'layer_name', 'param_mean', 'param_std', 'grad_mean', 'grad_std']].reset_index(drop=True)

    # If one dataframe has more rows, get the same number of rows from both dataframes
    layer_df2 = layer_df2[:len_df1].reset_index(drop=True)

    # Merge the columns of the two dataframes by joining on batch id
    diff_df = layer_df1.merge(layer_df2, how='inner', left_on=['batch_type', 'batch_id', 'layer_name'], right_on=['batch_type', 'batch_id', 'layer_name'], suffixes=('_1', '_2'))

    # Check that all rows got merged
    assert(len(diff_df) == len_df1)

    # Check that there is no difference between the param and grad values of both dataframes
    diff_df['diff_param'] = diff_df['param_mean_1'] - diff_df['param_mean_2']
    diff_df['diff_grad'] = diff_df['grad_mean_1'] - diff_df['grad_mean_2']
    unmatched_df = diff_df[(diff_df['diff_param'] != 0.0) | (diff_df['diff_grad'] != 0.0)]

    if (len(unmatched_df) != 0):
      print ('Unmatched')
    else:
      print ('Matched')

    return diff_df

### Debug Callback

In [ ]:
#export

#----------------------------------------------------
# This is a generic way to gather and report all metrics that are generated while running a model.
# It has three key components:
#   1. Collectors - any Callback, Hook or other component anywhere in the system can be 'probes' that collect data metrics
#        about any aspect of the system. This could be high-level metrics, or low-level debugging information. They send the
#        data they collect to the Debug Tracker
#   2. Tracker - this is really the heart of the system. It is the core component which consolidates all data from all
#        the various Collectors
#   3. Reporters - these display the collected data through different output mediums for easy visualisation and analysis.
#        eg. Tensorboard, Pandas, Pickle and potentially Matplotlib. This display could be in real-time while the run is in progress
#        or at the end, after the run has finished
#
# Data Metrics are gathered at five levels:
#   1. Run - this captures information relevant to the end-to-end run as a whole
#   2. Epoch - this represents an epoch.
#   3. Batch - this represents a mini-batch within an epoch. This captures batch-level information for all batches within
#        an epoch.
#   4. Step - this represents an intermediate operation (eg. forward, backward) during the execution of a batch. No metrics
#        tracked for the Step, it is only needed to group Layers at the next level.
#   5. Layer - this represents an internal data element of the model eg. a layer or composite layer, or a weight parameter
#
# These levels form a hierarchy, with each level being nested within the level above it. So, a Tracker captures multiple
# Runs, each of which contains multiple Epochs, each of which contains multiple Batches. Each Batch has multiple Steps, and 
# each Step has multiple Layers within it.
#
# In general, Run, Epoch and Batch normally capture high-level metrics that are 'externally' reported, while Step and Layer
# capture low-level metrics for debugging the internal working of the model
#
# This gives us one consistent mechanism for all metrics in the system, rather than have many different custom ways to 
# gather and report metrics eg. different debugging callbacks and hooks which gather different metrics for different models
# and then report them differently by plotting custom graphs, printing to the screening or via Tensorboard. We also have
# various other callbacks that do specific tasks like the MetricsCB, the Recorder that calculate different metrics 
# and statistics and then display them in different ways. Similarly the Optimiser has a custom method to print out
# hyper-parameter values for various parameter groups. All of these can be consolidated to work in a consistent way.
#
# It also enables us to save metrics from a run and load them later for analysis. We could also potentially compare
# metrics from one run with another, to examine trends and figure out if we're improving. Or even check whether two
# runs give us identical results as a debugging test to ensure that we haven't broken anything when making code changes
# to our application.
#----------------------------------------------------

#----------------------------------------------------
# The format of the data is as follows:
# Run:
#   runs = [run_one, run_two, run_three, ...], where each is a DebugRun object
#   run_one = {'run_id': 'Run-3 Mar 2020 5.14pm', 'num_epochs': 3, epochs=[epoch_one, epoch_two, ...]}
#   Each epoch is a DebugEpoch object
#   We could also track architecture settings 'hidden_sz': 1000, or 'graph': graph, for the model graph
# 
# Epoch:
#   epoch_one = {'epoch_id': 0, 'num_tr_batches': 75, 'num_val_batches': 75, batches=[batch_one, batch_two, ...]}
#   Each batch is a DebugBatch object
#   We could also track 'metrics': metrics
# 
# Batch:
#   batch_one = {'batch-id': 2-12, 'iter': 19, 'type': 'train', 'pred': 2.1982, 'loss': 0.0372, 'steps': [step_one, step_two, ...]}
#   Each step is a DebugStep object
#   We could also track 'hyper_params': hyper_params
#   eg. hyper_params = {'lr': [lr_gr1, lr_gr2, lr_gr3], 'mom': [mom_gr1, mom_gr2, mom_gr3]}
#   Validation batches we don't have to track param_steps, and rarely activation steps.
#
# Step:
#   step_one = {'step_id': 4, 'step_type': 'activation', 'step_operation': 'fwd', 'layers': [layer_one, layer_two, ...]}
#   step_two = {'step_id': 5, 'step_type': 'param', 'step_operation': 'discriminator loss', 'layers': [layer_one, layer_two, ...]}
#   Each layer is a DebugLayer object
#
# Layer:
#   For activation steps, used during both 'backwards' and 'optimisation'
#     layer_one = {'layer_name': '0.weight', 'data_mean': 0.234, 'data_std': 0.038, 'req_grad': True, 'grad_mean': 0.82, 'grad_std': 0.283}
#   For param steps, used during the 'forward' pass
#     layer_one = {'layer_name': 'input':, out_activation: 2.282}
#     layer_two = {'layer_name': '0':, out_activation: 2.282}
#     layer_three = {'layer_name': 'loss':, out_activation: 2.282}
#
# Results:
#   run_results = [row_one, row_two, ...]
#     Rows can be of different types, and are flattened ie. no nested sub-lists
#     Rows include the ID fields from all its parent objects eg. a batch row includes the 'run_id' and 'epoch_id' from 
#     its parent run and epoch respectively and a layer row includes the ID fields from its parent run, epoch, batch and step.
#       row_one = {'row_type': 'run', 'run_id': 'Run-3 Mar 2020 5.14pm', 
#                            'num_epochs': 3}
#       row_two = {'row_type': 'epoch', 'run_id': 'Run-3 Mar 2020 5.14pm', 
#                            'epoch_id': 0, 
#                            'num_tr_batches': 75, 'num_val_batches': 75}
#       row_three = {'row_type': 'batch', 'run_id': 'Run-3 Mar 2020 5.14pm', 'epoch_id': 0,
#                            'batch-id': 2-12, 'iter': 19, 'type': 'train', 
#                            'pred': 2.1982, 'loss': 0.0372}
#       row_four = {'row_type': 'layer', 'run_id': 'Run-3 Mar 2020 5.14pm', 'epoch_id': 0,
#                            'batch-id': 2-12, 'iter': 19, 'type': 'train', 
#                            'step_id': 4, 'step_type': 'activation', 'step_operation': 'fwd', 
#                            'layer_name': 'input':, out_activation: 2.282}
#
# Note that here we're choosing to group Layers (ie. data) by Step (ie. operation) since the same 'operation' can be 
# performed on many layers one after the other. In other words, we're sorting by operation first and data second. 
# However we may sometimes want to sort by data first and by operation second. eg. analyse all changes to 
# a layer activation, or a weight parameter as it proceeds across different operations. This can be easily done at reporting
# time. The Tensorboard reporting shows graphs by data anyway. And the Pandas results are flattened, allowing us to easily
# sort by operation or by data.
#----------------------------------------------------


#----------------------------------------------------
# Tracks information for a complete Run, consisting of mini-batches over a number of epochs
#----------------------------------------------------
class DebugRun():
  # List of fields which uniquely identify a Run row
  _id_names=['run_id']

  #----------------------------------------------------
  # Create an empty object with a unique run id.
  # Data is stored in a dictionary
  #----------------------------------------------------
  def __init__(self, num_epochs):
    timestr = time.strftime("%Y%m%d_%H%M%S")
    data = OrderedDict()
    data['run_id'] = f'Run-{timestr}'
    data['num_epochs'] = num_epochs
    data['epochs'] = []
    self.data = data

  #----------------------------------------------------
  # Update the information in my dictionary using the passed in data
  #----------------------------------------------------
  def update(self, run_data):
    assert(isinstance(run_data, dict))
    self.data.update(run_data)

  #----------------------------------------------------
  # Add an Epoch object to the run
  #----------------------------------------------------
  def add_epoch(self, epoch):
    assert(isinstance(epoch, DebugEpoch))
    self.data['epochs'].append(epoch)

  #----------------------------------------------------
  # The current epoch is the last epoch in our list
  #----------------------------------------------------
  def get_current_epoch(self):
    current_epoch_idx = len(self.data['epochs']) - 1
    return self.data['epochs'][current_epoch_idx]

  #----------------------------------------------------
  # Return the results at the end of the run. The hierarchy of
  # objects in the run are flattened into a flat list of rows, where each row 
  # is a dictionary. There is one row per object in the heirarchy, except for the
  # Step, whose row is merged into each Layer under it. Hence there are different
  # types of row - a run row, batch rows and combined step+layer rows - with different
  # fields for each type of row.
  #
  # Each row has some 'ID fields' and some 'data metric fields'. Note that these
  # are IDs in the database sense, which identify a unique row. eg. a Run carries 
  # a unique 'run_id' to identify it and also carries some 'data' fields such 
  # as the 'number of epochs'. The data metric fields are what we really care about
  # because they are values which we want to visualise and analyse.
  #
  # Similarly a Batch carries the 'batch_id', 'iteration number' and 'batch type' 
  # ID fields, which it then merges with the 'run_id' ID field of its parent Run
  # to uniquely identify it.
  #----------------------------------------------------
  def get_results(self):
    # Result is a list of rows
    all_rows = []

    # Prepare the Run row as a dictionary of all the non-list keys in the Run object
    # and add it to our result list 
    run_row = OrderedDict((k,v) for k,v in self.data.items() if not isinstance(v, list))
    run_row['row_type'] = 'run'
    all_rows.append(run_row)

    # Prepare a dictionary of all the 'unique ID fields' for a Run
    run_id_data = {k:self.data[k] for k in self._id_names}
    for epoch in self.data['epochs']:
      # Get the result rows for each epoch
      # Pass the Run ID fields to the Epoch object, so it can merge them with 
      # its own 'unique ID fields'.
      epoch_rows = epoch.get_results(run_id_data)
      # Add the epoch result rows to the result
      all_rows.extend(epoch_rows)

    return all_rows

  def __repr__(self):
    return f'{self.data}'

#----------------------------------------------------
# Tracks information for one epoch during a run
#----------------------------------------------------
class DebugEpoch():
  # List of fields which identify an Epoch row
  _id_names=['epoch_id']

  #----------------------------------------------------
  # Create an empty object with a unique epoch id.
  # Data is stored in a dictionary
  #----------------------------------------------------
  def __init__(self, i_epoch):
    data = OrderedDict()
    # Concat the epoch number and batch number to generate a batch id
    data['epoch_id'] = i_epoch
    data['num_tr_batches'] = 0
    data['num_val_batches'] = 0
    data['batches'] = []
    self.data = data

  #----------------------------------------------------
  # Update the information in my dictionary using the passed in data
  #----------------------------------------------------
  def update(self, epoch_data):
    assert(isinstance(epoch_data, dict))
    self.data.update(epoch_data)

  #----------------------------------------------------
  # Add a Batch object to the epoch
  #----------------------------------------------------
  def add_batch(self, batch):
    assert(isinstance(batch, DebugBatch))
    self.data['batches'].append(batch)

  #----------------------------------------------------
  # The current batch is the last batch in our list
  #----------------------------------------------------
  def get_current_batch(self):
    current_batch_idx = len(self.data['batches']) - 1
    return self.data['batches'][current_batch_idx]

  #----------------------------------------------------
  # Return the results for epoch, as a list of one epoch row, and all the 
  # batch rows within that epoch
  #----------------------------------------------------
  def get_results(self, run_id_data):
    # Result is a list of rows
    epoch_rows = []

    # Prepare the data for the epoch as a dictionary of all the non-list keys in the Epoch object
    epoch_data = OrderedDict((k,v) for k,v in self.data.items() if not isinstance(v, list))
    # Create an epoch row by merging the Run's ID fields with the epoch data
    epoch_row = {**run_id_data, **epoch_data}
    epoch_row['row_type'] = 'epoch'
    # and add it to our result list 
    epoch_rows.append(epoch_row)

    # Prepare a dictionary of all the 'unique ID fields' for an Epoch
    # Then merge it with the Run ID fields
    epoch_id_data = {k:self.data[k] for k in self._id_names}
    run_epoch_id_data = {**run_id_data, **epoch_id_data}

    for batch in self.data['batches']:
      # Get the result rows for each batch
      # Pass the Run and Epoch ID fields to the Batch object, so it can merge them with 
      # its own 'unique ID fields'.
      batch_rows = batch.get_results(run_epoch_id_data)
      # Add the batch result rows to the result
      epoch_rows.extend(batch_rows)

    return epoch_rows

#----------------------------------------------------
# Tracks information for one mini-batch during a run
#----------------------------------------------------
class DebugBatch():
  # List of fields which identify a Batch row
  _id_names=['batch_id', 'iter', 'batch_type']

  #----------------------------------------------------
  # Create an empty object with a unique batch id.
  # Data is stored in a dictionary
  #----------------------------------------------------
  def __init__(self, batch_type, i_epoch, i_batch, i_iter):
    data = OrderedDict()
    # Concat the epoch number and batch number to generate a batch id
    data['batch_id'] = f'{i_epoch}-{i_batch}'
    data['iter'] = i_iter
    data['batch_type'] = batch_type
    data['pred'] = 0.
    data['loss'] = 0.
    data['steps'] = []
    self.data = data

  #----------------------------------------------------
  # 
  #----------------------------------------------------
  def _flatten (self, data, flat_data):
    for k, v in data.items():
      if (isinstance(v, dict)):
        flatten(v, flat_data)
      elif (isinstance(v, list) and (len(v) == 1)):
        flat_data.update({k: v[0]})
      elif (isinstance(v, list) and (len(v) > 1)):
        flat_data.update({f'{k}{i}': vi for i, vi in enumerate(v)})
      else:
        flat_data.update({k: v})

  #----------------------------------------------------
  # Update the information in my dictionary using the passed in data
  #----------------------------------------------------
  def update(self, batch_data):
    assert(isinstance(batch_data, dict))

    flat_data = {}
    self._flatten(batch_data, flat_data)

    self.data.update(flat_data)

  #----------------------------------------------------
  # Add a Step object to the batch
  #----------------------------------------------------
  def add_step(self, step_data):
    steps = self.data['steps']
    steps.append(step_data)

  #----------------------------------------------------
  # The current step is the last step in our list. Except for a
  # newly created batch for which no step has been added yet.
  #----------------------------------------------------
  def get_current_step(self):
    num_steps = len(self.data['steps'])
    if (num_steps == 0):
      # No steps have been added yet, so one will have to be created
      return None
    else:
      current_step_idx = num_steps - 1
      return self.data['steps'][current_step_idx]

  #----------------------------------------------------
  # Return the results for a batch, as a list of one batch row, and all the 
  # step rows within that batch
  #----------------------------------------------------
  def get_results(self, run_epoch_id_data):
    # Result is a list of rows
    batch_rows = []

    # Prepare the data for the batch as a dictionary of all the non-list keys in the Batch object
    batch_data = OrderedDict((k,v) for k,v in self.data.items() if not isinstance(v, list))
    # Create a batch row by merging the Run's and Epoch's ID fields with the batch data
    batch_row = {**run_epoch_id_data, **batch_data}
    batch_row['row_type'] = 'batch'
    # and add it to our result list 
    batch_rows.append(batch_row)

    # Prepare a dictionary of all the 'unique ID fields' for a Batch
    batch_id_data = {k:self.data[k] for k in self._id_names}

    # Go through every Step, and every Layer in each Step in the batch and
    # prepare one row per Layer. We don't create a separate row for Steps, as it
    # doesn't have any 'data metric' fields.
    for step in self.data['steps']:
      # Prepare a dictionary of all the 'unique ID fields' for a Step
      step_id_data = {k:step.data[k] for k in step._id_names}
      for layer in step.data['layers']:
        layer_data = layer.data
        # Create a layer row by merging the Run and Epoch ID fields, Batch ID fields and 
        # Step ID fields with the Layer data
        layer_row = {**run_epoch_id_data, **batch_id_data, **step_id_data, **layer_data}
        layer_row['row_type'] = 'layer'
        batch_rows.append(layer_row)

    return batch_rows

  def __repr__(self):
    return f'{self.data}'

#----------------------------------------------------
# Tracks information for one operation step within a batch
# A 'step' is really any intermediate operation that you want to track during a batch. Think of it as 
# a 'milestone' or 'point in time' during the execution of a batch where you want to track the data 
# values of the layers of your model. So you can have as many as you want at any point in the training
# loop. The 'step_operation' is a name to identify that step and can be any label that you choose.
#
# The typical steps are 'fwd', 'bkwd' and 'opt' for the forward pass, backward pass and optimiser update 
# operations respectively. However, you can include any other steps that you need for more complicated 
# training loops eg. for a GAN where you can have multiple backward passes or optimisation passes 
# within the same batch. So we can choose a distinct name for each operation to identify it.
#
# Steps are of two types, based on the type of data that they track viz. 'activation' to track the
# output activation of a layer, and 'param' to track the parameter values and gradients of weight
# and bias parameters.
#
# The 'step_id' is an integer number that is auto calculated and keeps incrementing every time a new
# step is tracked, and is unique for a run, across all batches of all epochs. You can use it as 
# your 'timestep' on the X-axis while graphing the data values as they change during the execution
# of a run. There are two sequences of step_id, one for the 'activation' steps and another for the
# 'param' steps
#----------------------------------------------------
class DebugStep():
    # List of fields which identify a Step row
  _id_names=['step_id', 'step_operation', 'step_type']

  #----------------------------------------------------
  # Create an empty object with a unique step id, operation and
  # step type. Data is stored in a dictionary
  #----------------------------------------------------
  def __init__(self, step_id, step_operation, step_type):
    data = dict()
    data['step_id'] = step_id
    data['step_operation'] = step_operation
    data['step_type'] = step_type
    data['layers'] = []
    self.data = data

  #----------------------------------------------------
  # Add a Layer object to the step
  #----------------------------------------------------
  def add_layer(self, layer_data):
    layers = self.data['layers']
    layers.append(layer_data)

  #----------------------------------------------------
  # Does the given 'step_type' and 'step_operation' match my step type and
  # operation?
  #----------------------------------------------------
  def is_same_step(self, step_type, step_operation):
    return ((self.data['step_type'] == step_type) and
            (self.data['step_operation'] == step_operation))

#----------------------------------------------------
# Tracks information for one data element (either a layer or a parameter), as part of
# an operation step within a batch.
#
# For instance, a 'forward' step involves multiple sequential layers each with its
# own activation. Similarly a 'backward' step involves multiple weight and bias
# parameters, each with its own parameter value and gradient.
#----------------------------------------------------
class DebugLayer():
  # List of fields which identify a Step row
  _id_names=['layer_name']

  #----------------------------------------------------
  # Create a populated object with a layer name and data values. 
  # Data is stored in a dictionary
  #----------------------------------------------------
  def __init__(self, step_type, layer_name, layer_value, **kwargs):
    data = dict()
    data['layer_name'] = layer_name
    if (step_type == 'activation'):
      out_activation = layer_value
      data['out_activation'] = out_activation.mean().item()
    else:
      param = layer_value
      data['param_mean'] = param.data.mean().item()
      data['param_std'] = param.data.std().item()
      data['requires_grad'] = param.requires_grad
      if (param.grad is not None):
        data['grad_mean'] = param.grad.mean().item()
        data['grad_std'] = param.data.std().item()

    if (kwargs is not None):
      data.update(kwargs)
    self.data = data

#----------------------------------------------------
# 
# It is both a Callback as well as a regular class with additional methods that expose its own API.
# In other words, it implements several of the Callback events as part of the training loop and also 
# has independent methods in its API which can be called directly by other components.
#
# These other components would usually be Callbacks that collect data metrics while processing their
# own Callback events. They then use the DebugTrackers API to send their metrics to the DebugTracker
# However, these components could also be Hooks which are triggered outside of a Callback event
# and could send data to the DebugTracker to consolidate.
#
# Hence the Tracker works in conjunction with these other Callbacks and Hooks. By itself it
# doesn't track much metric data of its own.
#
# The Tracker uses its Callback events to initialise the upper levels of the data hierarchy,
# by creating a Run object at the beginning of the run, and Batch objects at the beginning of
# each training and validation batch. This creates the 'skeleton' of the data and keeps it ready
# but without any useful metrics to be tracked. Those metrics start getting added as various 
# Callbacks or Hooks start sending in their data via the Tracker API. They might update 
# metrics for the existing Run and Batch, or they might add Steps and Layers under the 
# Batches. Obviously they cannot add new Runs or Batches via the API, as those are enforced 
# within the Tracker via Callback events only.
#
# The API allows you to add a Layer but doesn't allow you to add a Step explicitly. The
# Step gets added automatically by the Tracker whenever it is asked to add a Layer for 
# an operation that is different from the previous operation.
#
# Since everything is happening sequentially in time as we execute the training loop, the Tracker
# has a sense of the Current Run, the Current Batch within it and the Current Step within
# that. The Callback and Hook callers don't need to keep track of any of that state. They 
# do not need to know what the id of the current run is, or the iteration number of 
# the current batch and so on, as those are internally managed, assigned and incremented 
# by the Tracker. This simplifies the work of the callers because they can simply send any
# new data metric they gather and the Tracker adds it to the right Run, Data or Step object
# in the heirarchy. This also means that the API doesn't allow you to go back and update
# previous Runs, Batches or Steps. 
# 
# Also note there is no notion of the Current Layer, so you can only add new Layers, 
# but cannot update an existing Layer. This is because a Layer has meaning only in 
# the context of an operation (ie. a Step) which is really just a distinct name for a 
# point in time within the execution of a batch. So it doesn't make sense to modify 
# a previous point in time. Callers are free to define another operation with 
# distinct labels to reflect a subsequent point in time. Since each Layer corresponds
# to a specific data element such as a layer or a weight parameter, it is easy to
# trace the state changes of that data element chronologically over the course of 
# multiple operations.
#
# In order to control how much data we collect and track, we can define some
# simple filter criteria to select the batches and steps for which we track data. The
# 'freq' lets us skip batches eg. 'freq=5' tracks data for 1 out of every 5 batches.
# 'max_count' is the maximum index number of the batch within an epoch. 
# eg. 'max_count=3' will not track anything after the first 3 batches in each epoch.
# Even for the first 3 batches, it applies the 'freq' to decide what to skip.
# 'data_prefixes' is a list of prefix substrings to select the layer names.
# eg. 'data_prefixes=['G_AB', 'D_AB']' tracks only those layers whose names begin
# with either "G_AB*" or "D_AB+"
#
# The 'disp' is a tuple of two booleans used to turn on the Tensorboard and
# Pandas reporters for displaying results.
#----------------------------------------------------
class DebugTracker(Callback):
  def __init__(self, freq=1, max_count=5, data_prefixes=None, disp=(False, False)):
    self.runs = []
    self.freq = freq
    self.max_count = max_count
    self.data_prefixes = data_prefixes
    # 'do' is True when the current batch is to be tracked
    self.do = True
    self.do_epoch = True
    self.set_display(disp)

  #----------------------------------------------------
  # Start a run at the beginning of the training loop
  #----------------------------------------------------
  def begin_fit(self, ctx):
    # Create a new Run object
    new_run = self._create_run(ctx.num_epochs)
    # Counter for validation iterations
    self.val_iter = -1
    if (self.tb is not None):
      # Initialise Tensorboard for the run
      run_name = new_run.data['run_id']
      self.tb.start_run(f'{run_name}')

  #----------------------------------------------------
  # Create an Epoch object at the start of a new epoch
  #----------------------------------------------------
  def begin_epoch(self, ctx):
    if (self._track_epoch(ctx.i_epoch)):
      self._create_epoch(ctx.i_epoch)

  #----------------------------------------------------
  # Update the number of training batches at the start of training batches within
  # the epoch
  #----------------------------------------------------
  def begin_tr(self, ctx):
    epoch_data = {}
    epoch_data['num_tr_batches'] = ctx.num_batches
    self.update_epoch(epoch_data)

  #----------------------------------------------------
  # Create a Batch object at the start of a new training batch
  #----------------------------------------------------
  def begin_tr_batch(self, ctx):
    if (self._track_batch(ctx.i_epoch, ctx.i_batch)):
      self._create_batch('train', ctx.i_epoch, ctx.i_batch, ctx.i_tr_iter)

  #----------------------------------------------------
  # Update the number of validation batches at the start of validation batches
  # within the epoch
  #----------------------------------------------------
  def begin_val(self, ctx):
    epoch_data = {}
    epoch_data['num_val_batches'] = ctx.num_batches
    self.update_epoch(epoch_data)

  #----------------------------------------------------
  # Create a Batch object at the start of a new validation batch
  #----------------------------------------------------
  def begin_val_batch(self, ctx):
    if (self._track_batch(ctx.i_epoch, ctx.i_batch)):
      # The Trainer does not need to track validation iterations, so 'ctx' has no
      # counter for it. We need it only to provide an incremental X-axis 'step' for 
      # plotting the Tensorboard graph of validation loss metrics. This counter is
      # not a true count of every validation iteration, but only those iterations that
      # the DebugTracker tracks 
      self.val_iter += 1
      self._create_batch('val', ctx.i_epoch, ctx.i_batch, self.val_iter)

  #----------------------------------------------------
  # Finish a Run at the end of the training loop, and report results
  #----------------------------------------------------
  def end_fit(self, ctx):
    if (self.tb is not None):
      # Report results to Tensorboard and close the connection
      self._tb_results()
      self.tb.end_run()

  #----------------------------------------------------
  # Update metrics for a run
  #----------------------------------------------------
  def update_run(self, run_data):
    current_run = self._get_current_run()
    current_run.update(run_data)

  #----------------------------------------------------
  # Update metrics for an epoch
  #----------------------------------------------------
  def update_epoch(self, epoch_data):
    current_run = self._get_current_run()
    # !!!!!! do we need a separate self.do_epoch or use a single self.do?
    if (self.do_epoch):
      assert(isinstance(epoch_data, dict))
      current_epoch = self._get_current_epoch()
      current_epoch.update(epoch_data)

  #----------------------------------------------------
  # Update metrics for a batch
  #----------------------------------------------------
  def update_batch(self, batch_data):
    if (self.do):
      assert(isinstance(batch_data, dict))
      current_batch = self._get_current_batch()
      current_batch.update(batch_data)

  #----------------------------------------------------
  # Add a new 'param' layer 
  #----------------------------------------------------
  def add_param_layer(self, step_operation, param_name, param, **kwargs):
    # Check if the batch and layer is to be tracked
    if (self.do and self._track_layer(param_name)):
      # Create a Layer. Internally it automatically creates a new Step if the
      # operation is different from the previous one
      step_type = 'param'
      self._create_layer(step_type, step_operation, param_name, param, **kwargs)
 
      # Report histogram data to Tensorboard in real-time. Since tensors can be
      # quite large, we track only scalar metrics by using the mean and std.
      # However we can send the tensor's histogram to Tensorboard while the run is
      # in progress without saving it till the end of the run for result processing 
      if (self.tb is not None):
        # Report the parameter data value and gradient for the current step_id
        data_name, grad_name = param_name + '/data', param_name + '/grad'
        step_id = self.step_id[step_type]
        self.tb.add_hist(data_name, param.data, step_id)
        if (param.grad is not None):
          self.tb.add_hist(grad_name, param.grad, step_id)

  #----------------------------------------------------
  # Add a new 'activation' layer 
  #----------------------------------------------------
  def add_activation_layer(self, step_operation, data_layer, out_activation, **kwargs):
    # Check if the batch and layer is to be tracked
    if (self.do and self._track_layer(data_layer)):
      # Create a Layer. Internally it automatically creates a new Step if the
      # operation is different from the previous one
      step_type = 'activation'
      self._create_layer(step_type, step_operation, data_layer, out_activation, **kwargs)

      # Report histogram data to Tensorboard in real-time. Since tensors can be
      # quite large, we track only scalar metrics by using the mean and std.
      # However we can send the tensor's histogram to Tensorboard while the run is
      # in progress without saving it till the end of the run for result processing 
      if (self.tb is not None):
        # Report the activation value for the current step_id
        data_name = data_layer + '/activation'
        step_id = self.step_id[step_type]
        self.tb.add_hist(data_name, out_activation, step_id)

  #----------------------------------------------------
  # Initialise the Tensorboard and Pandas Reporting objects
  #----------------------------------------------------
  def set_display(self, disp):
    assert(isinstance(disp, tuple) and (len(disp) == 2))
    assert(isinstance(disp[0], bool) and isinstance(disp[1], bool))
    disp_tb, disp_pd = disp
    self.tb = DebugTensorboard() if disp_tb else None
    self.pd = DebugPandas() if disp_pd else None

  #----------------------------------------------------
  # Get the flattened result rows for a run
  #----------------------------------------------------
  def _get_results(self, run_idx=-1):
    run = self._get_run(run_idx)
    all_rows = run.get_results()
    return all_rows

  #----------------------------------------------------
  # Get the result rows for a run, and display the scalar values on Tensorboard
  #----------------------------------------------------
  def _tb_results(self):
    all_rows = self._get_results()

    for row_data in all_rows:
      row_type = row_data['row_type']

      if (row_type == 'layer'):
        layer_row_data = row_data
        layer_id_names = DebugRun._id_names + DebugEpoch._id_names + DebugBatch._id_names + DebugStep._id_names + DebugLayer._id_names
        layer_metric_data = {k:v for k,v in layer_row_data.items() if ((k not in layer_id_names) and isinstance(v, float))}
        for k, v in layer_metric_data.items():
          tag = layer_row_data['layer_name'] + '/' + k
          self.tb.add_scalar(tag, v, layer_row_data['step_id'])

      elif (row_type == 'batch'):
        batch_row_data = row_data
        batch_id_names = DebugRun._id_names + DebugEpoch._id_names + DebugBatch._id_names
        batch_metric_data = {k:v for k,v in batch_row_data.items() if ((k not in batch_id_names) and isinstance(v, float))}
        for k, v in batch_metric_data.items():
          tag = batch_row_data['batch_type'] + '/' + k
          self.tb.add_scalar(tag, v, batch_row_data['iter'])
      
      elif (row_type == 'epoch'):
        epoch_row_data = row_data
        epoch_id_names = DebugRun._id_names + DebugEpoch._id_names
        epoch_metric_data = {k:v for k,v in epoch_row_data.items() if ((k not in epoch_id_names) and isinstance(v, float))}
        for k, v in epoch_metric_data.items():
          tag = 'epoch' + '/' + k
          self.tb.add_scalar(tag, v, epoch_row_data['epoch_id'])

      else:
        pass

  #----------------------------------------------------
  # Show results on Tensorboard for a previously saved run that has been loaded. 
  # Do not use this for results of 'live' runs as they are automatically 
  # displayed on Tensorboard
  #----------------------------------------------------
  def tb_saved_results(self, run_idx=-1):
    if (self.tb is not None):
      # Start Tensorboard for reporting results for a run
      run = self._get_run(run_idx)
      run_name = run.data['run_id']
      self.tb.start_run(f'{run_name}')

      # Report results to Tensorboard and close the connection
      self._tb_results()
      self.tb.end_run()

  #----------------------------------------------------
  # Get the result rows for a run, and format them as a Pandas dataframe
  #----------------------------------------------------
  def pd_results(self):
    assert(self.pd is not None)
    all_rows = self._get_results()
    res_dfs = self.pd.create_dfs (all_rows)

    return res_dfs

  #----------------------------------------------------
  # Save the Run object and its hierarchy to a pickle file
  #----------------------------------------------------
  def save_run(self, save_path, run_idx=-1):
    run = self._get_run(run_idx)
    with open(save_path, 'wb') as sf:
      pickle.dump(run, sf)

  #----------------------------------------------------
  # Load a pickle dump of a previously saved Run object
  #----------------------------------------------------
  def load_run(self, save_path):
    with open(save_path, 'rb') as sf:
      run = pickle.load(sf)
    self.runs.append(run)

  #----------------------------------------------------
  # Create a new Run object
  #----------------------------------------------------
  def _create_run(self, num_epochs):
    run_idx = len(self.runs)
    new_run = DebugRun(num_epochs)
    self.runs.append(new_run)

    # Reset the global step_id at the beginning of the run. Step IDs will start
    # from 0 and increment for every new step for the duration of the run
    self.step_id = {'activation': -1, 'param': -1}

    return new_run

  #----------------------------------------------------
  # Create a new Epoch object and add it to the current run
  #----------------------------------------------------
  def _create_epoch(self, i_epoch):
    new_epoch = DebugEpoch(i_epoch)
    current_run = self._get_current_run()
    current_run.add_epoch(new_epoch)

  #----------------------------------------------------
  # Create a new Batch object and add it to the current run
  #----------------------------------------------------
  def _create_batch(self, batch_type, i_epoch, i_batch, i_iter):
    new_batch = DebugBatch(batch_type, i_epoch, i_batch, i_iter)
    current_epoch = self._get_current_epoch()
    current_epoch.add_batch(new_batch)

  #----------------------------------------------------
  # Create a new Step object and add it to the current batch
  #----------------------------------------------------
  def _create_step(self, step_id, step_operation, step_type):
    new_step = DebugStep(step_id, step_operation, step_type)
    current_batch = self._get_current_batch()
    current_batch.add_step(new_step)
    return new_step

  #----------------------------------------------------
  # First check if a new Step object has to be created because the operation 
  # has changed from the previous. If not, get the current step.
  # Then create a new Layer object and add it to the step as above.
  #----------------------------------------------------
  def _create_layer(self, step_type, step_operation, layer_name, layer_value, **kwargs):
    new_layer = DebugLayer(step_type, layer_name, layer_value, **kwargs)
    current_step = self._get_step(step_type, step_operation)
    current_step.add_layer(new_layer)

  #----------------------------------------------------
  # Get the current step and check if the operation is the same as the previous.
  # If it is, return the current step. If it isn't, create a new step.
  #----------------------------------------------------
  def _get_step(self, step_type, step_operation):
    current_step = self._get_current_step()
    if ((current_step is not None) and
        current_step.is_same_step(step_type, step_operation)):
      # current_step is None if this is a new batch with no steps so far
      return current_step
    else:
      # Operation has changed, or this is a new batch with no steps so far.
      # In either case we need to create a new step after incrementing our
      # global step_id
      self.step_id[step_type] += 1
      step_id = self.step_id[step_type]
      new_step = self._create_step(step_id, step_operation, step_type)
      return new_step

  #----------------------------------------------------
  # A new run object is automatically created and added to the list when the
  # training loop starts. Therefore, the last added run object in the list 
  # is the current run
  #----------------------------------------------------
  def _get_current_run(self):
    current_run_idx = len(self.runs) - 1
    return self._get_run(current_run_idx)

  #----------------------------------------------------
  # Get a run object, given its index in the list
  #----------------------------------------------------
  def _get_run(self, run_idx):
    return self.runs[run_idx]

  #----------------------------------------------------
  # A new epoch object is automatically created and added to the list when an
  # epoch starts. Therefore, the last added epoch object in the list is the current epoch
  #----------------------------------------------------
  def _get_current_epoch(self):
    current_run = self._get_current_run()
    current_epoch = current_run.get_current_epoch()
    return current_epoch

  #----------------------------------------------------
  # A new batch object is automatically created and added to the list when the
  # training or validation batch starts. Therefore, the last added batch object
  # in the list is the current batch
  #----------------------------------------------------
  def _get_current_batch(self):
    current_epoch = self._get_current_epoch()
    current_batch = current_epoch.get_current_batch()
    return current_batch

  #----------------------------------------------------
  # The last added step in the list is the current step
  #----------------------------------------------------
  def _get_current_step(self):
    current_batch = self._get_current_batch()
    current_step = current_batch.get_current_step()
    return current_step

  #----------------------------------------------------
  # Return True if this epoch is to be tracked based on the
  # filter criteria
  # !!!!!!!!!!! Implement the filter criteria
  #----------------------------------------------------
  def _track_epoch(self, i_epoch):
    self.do_epoch = True
    return self.do_epoch

  #----------------------------------------------------
  # Return True if this batch is to be tracked based on the
  # filter criteria
  #----------------------------------------------------
  def _track_batch(self, i_epoch, i_batch):
    self.do = ((i_batch % self.freq == 0) and (i_batch < self.max_count))
    return self.do

  #----------------------------------------------------
  # Return True if this layer is to be tracked based on the
  # filter criteria
  #----------------------------------------------------
  def _track_layer(self, data_name):
    do_layer = (self.data_prefixes is None) or (any(data_name.startswith(data_prefix) for data_prefix in self.data_prefixes))
    return do_layer

  #----------------------------------------------------
  # !!!!!! TODO
  #----------------------------------------------------
  def _print_kw(self, kw):
    res = ''
    for run in self.runs:
      for k, v in run.data.items():
        if (isinstance(v, list)):
          res += f'\t\t{k}:' + '\n'.join([f'{v}' for i in v])
        else:
          res += f'{k}:{v}'
    return res

  #----------------------------------------------------
  # !!!!!! TODO
  #----------------------------------------------------
  def __repr__(self):
    runs=[f'{run.data}' for run in self.runs]
    batches=[f'{batch.data}' for run in self.runs for batch in run.data['batches']]
    return ('\n'.join(runs + batches))

  def temp__repr__(self):
    return f'{self.runs}'

In [ ]:
dtr = DebugTracker(disp=(False, True))
dtr._create_run(2)
dtr.update_run({'setting': .009})
dtr._create_epoch(0)
dtr.update_epoch({'num_tr_batches': 50, 'metric': 7.7})
dtr._create_batch('train', 0, 0, 0)
dtr.update_batch({'pred': 2.1982, 'loss': 0.0372})
dtr.add_activation_layer('fwd', 'inp', tensor([2., 3.]))
dtr.add_activation_layer('fwd', 'layer 0', tensor([5., 9.]), extra=9, info=46.)
dtr._get_results()

In [ ]:
_, _, _, dtrdf = dtr.pd_results()
dtrdf

,run_id,num_epochs,setting,row_type,epoch_id,num_tr_batches,num_val_batches,metric,batch_id,iter,batch_type,pred,loss,step_id,step_operation,step_type,layer_name,out_activation,extra,info
0,Run-20200717_044044,2.0,0.009,run,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Run-20200717_044044,NaN,NaN,epoch,0.0,50.0,0.0,7.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Run-20200717_044044,NaN,NaN,batch,0.0,NaN,NaN,NaN,0-0,0.0,train,2.1982,0.0372,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Run-20200717_044044,NaN,NaN,layer,0.0,NaN,NaN,NaN,0-0,0.0,train,NaN,NaN,0.0,fwd,activation,inp,2.5,NaN,NaN
4,Run-20200717_044044,NaN,NaN,layer,0.0,NaN,NaN,NaN,0-0,0.0,train,NaN,NaN,0.0,fwd,activation,layer 0,7.0,9.0,46.0


In [ ]:
#export

#----------------------------------------------------
# Debug Callback to print yhat and loss after each batch
#----------------------------------------------------
class DebugYhatLossCB(Callback):
  def __init__(self, fwd=True, bkwd=False, prefix='\t'):
    self.fwd, self.bkwd = fwd, bkwd
    self.bkwd_pre = False # Don't show gradients values before backward()
    self.prefix = prefix

  def begin_tr(self, ctx):
    if (self.fwd):
      # Print Headers
      print (f'{self.prefix}Total {ctx.num_epochs} epochs with {ctx.num_batches} batches')
      print (f'{self.prefix}Batch, Pred, Loss')

  def after_tr_loss(self, ctx):
    with torch.no_grad():
      dtr = ctx.dtr
      if (dtr):
        dtr.update_batch({'pred': ctx.yhat.mean().item(), 'loss': ctx.loss.item()})

      if (self.fwd):
        print (f'{self.prefix}{ctx.i_epoch:2}.{ctx.i_batch}, {ctx.yhat.mean():.4f}, {ctx.loss:.4f}')

      #if (self.bkwd_pre):
        #self.print_grad (ctx.model, before='Pre Gradients')

  def after_val_loss(self, ctx):
    dtr = ctx.dtr
    if (dtr):
      dtr.update_batch({'pred': ctx.yhat.mean().item(), 'loss': ctx.loss.item()})

  def after_tr_backward(self, ctx):
    with torch.no_grad():
      if (self.bkwd and ctx.dtr):
        dtr = ctx.dtr
        for name, param in ctx.model.named_parameters():
          dtr.add_param_layer('bkwd', name, param)

      #self.print_grad (ctx.model)

  def end_tr_batch(self, ctx):
    if (ctx.dtr):
      #lrs = [pg['lr'] for pg in ctx.opt.param_groups]
      #moms = [pg['betas'][0] for pg in ctx.opt.param_groups]
      #ctx.dtr.update_batch({'lrd': lrs, 'momd': moms})
      hps = {hp:[pg[hp] for pg in ctx.opt.param_groups] for hp in ctx.opt.param_groups[0].keys() if hp != 'params'}
      hps = self.format_hp(hps)
      ctx.dtr.update_batch(hps)

  @staticmethod
  def format_hp(hps):
    new_hps = {}
    for hp, hp_val in hps.items():
      if (isinstance(hp_val[0], tuple)):
        assert(len(hp_val[0]) == 2)
        hp1, hp2 = f'{hp}_1', f'{hp}_2'
        if (hp == 'betas'):
          hp1 = 'mom'
        hp_val1, hp_val2 = zip(*hp_val)
        new_hps[hp1] = list(hp_val1)
        new_hps[hp2] = list(hp_val2)
      else:
        new_hps[hp] = hp_val
    return new_hps

  def print_grad(self, model, before='\t\t'):
    hdr_name = 'Name'
    print (f'{before}{hdr_name:12}: Shape, Requires, Leaf, Grad')
    for name, param in model.named_parameters():
      print (f'{before}{name:12}: {tuple(param.data.size())}, {param.requires_grad}, {param.is_leaf}, {param.grad.mean() if param.grad is not None else 0:0.4G}')


In [ ]:
dtr = DebugTracker(freq=2, data_prefixes=['2.'], disp=(True, True))
debug_cbs = [dtr, DebugYhatLossCB(bkwd=True)]
debug_loop = run_mnist(debug_cbs, num_epochs=2, repro=True, dtr=dtr)

#dtr.tb_results()
dcdrrun_df, dcdrbatch_df, dcdrlayer_df, dcdrdf = dtr.pd_results()

In [ ]:
dcdrdf

In [ ]:
dcdrrun_df

,run_id,num_epochs,row_type,epoch_id,num_tr_batches,num_val_batches
0,Run-20200717_044044,2.0,run,NaN,NaN,NaN
1,Run-20200717_044044,NaN,epoch,0.0,98.0,10.0
14,Run-20200717_044044,NaN,epoch,1.0,98.0,10.0


In [ ]:
dcdrbatch_df

In [ ]:
dcdrlayer_df

**Set up Tensorboard**

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir tbtry

In [ ]:
!rm -R tbtry
!ls -lR tbtry

### Export

In [ ]:
!wget https://raw.githubusercontent.com/ketanhdoshi/ml/master/lib/nb_export.py

In [ ]:
from nb_export import notebook2scriptSingle
notebook2scriptSingle(gn_path + '/lib/training_lib.ipynb', gn_path + '/exp')

Converted gdrive/My Drive/Colab Notebooks/lib/training_lib.ipynb to gdrive/My Drive/Colab Notebooks/exp/nb_training.py


### Junk

In [ ]:
class MetricsGrp(Callback):
  def __init__(self, metrics):
    self.metrics = metrics
    # !!!!! Register callbacks for each metric

  # ----------------------------
  # Print headers for all the result columns
  # ----------------------------   
  def begin_fit(self, ctx):
    # ['epoch', 'tr_loss', 'smooth_loss', 'val_loss', 'metric1', 'metric2', ..'time']
    headers = ['epoch']
    for metric in self.metrics: headers += metric.header
    headers += ['time']
    ctx.logger(headers)

  # ----------------------------
  # Track start time at the beginning of each epoch
  # ----------------------------   
  def begin_epoch(self, ctx):
    ctx.start_time = time.time()

  # ----------------------------
  # Print results at the end of each epoch
  # ----------------------------   
  def end_epoch(self, ctx):
    # Flattened metric results and epoch time taken
    values = [results for metric in self.metrics for results in metric._results()]
    epoch_time = [format_time(time.time() - ctx.start_time)]
    #for metric in self.metrics: values += metric._results()

    # Epoch number, formatted metric results, and epoch time
    res = [str(ctx.i_epoch)]
    res += [f'{val:.6f}' for val in values]
    res += epoch_time

    ctx.logger(res)

    # Report metrics to the DTR
    if (ctx.dtr): self._report(ctx.dtr, values + epoch_time)

  # ----------------------------
  # Report metrics to the DTR
  # ----------------------------   
  def _report(self, dtr, results):
    headers = [header for metric in self.metrics for header in metric.header] + ['time']
    epoch_metrics = {header:result for header, result in zip(headers, results)}
    dtr.update_epoch(epoch_metrics)

#----------------------------------------------------
# Base class for all Metrics callbacks
#
# It handles some common functionality such as managing the metric result value (one or more)
# that it computes per epoch. It resets that value at the
# end of each epoch. It provides abstract methods to do any necessary computations
# at the end of each batch and at the end of each epoch. The child class provides
# implementations for these methods. It also accumulates the inputs (ie. preds and targs)
# after each batch, which the child class can use for its end-epoch calculations
# if it needs.
#----------------------------------------------------
class MetricsBaseCB(Callback):
  header = ['metric']
  # Accumulated 'yhat' predictions and 'yb' targets. These are singletons which
  # are common for all child CBs. Treated as read-only by all child CBs
  accumulate = False
  yhat, yb = None, None
  last_accumulated_batch = -1

  def __init__(self):
    self.num_value = len(self.header)

  # ----------------------------
  # Reset any state, when training starts and at beginning of each epoch
  # ----------------------------   
  def _reset(self, begin_fit=False):
    pass

  # ----------------------------
  # Return the metric values
  # ----------------------------   
  def _results(self):
    values = [self.value] if (self.num_value == 1) else self.value
    values = [val.item() if (isinstance(val, torch.Tensor)) else val for val in values]
    return values

  # ----------------------------
  # Do any required calculations at the end of the batch. A flag indicates if it
  # is a training or validation batch.
  # Functionality gets implemented within the child callback.
  # ----------------------------   
  def _calc_batch(self, ctx, is_val=True):
    pass

  # ----------------------------
  # Reset the accumulated values
  #
  # Note that since these attributes are singletons, they must be modified using
  # 'MetricsBaseCB.<attr>'. Modifying them via 'cls.<attr>' modifies them only
  # for the child class, since 'cls' points to the child and not the base. Similarly
  # modfifying them via 'self.<attr>' modifies them only for the child instance.
  # However, to get the value they can be referenced by any of the above ways ie.
  # 'cls.<attr>', 'self.<attr>' or 'MetricsBaseCB.<attr>' will all work.
  # ----------------------------   
  @classmethod
  def _reset_accumulate(cls):
    MetricsBaseCB.yhat, MetricsBaseCB.yb = None, None
    MetricsBaseCB.last_accumulated_batch = -1

  # ----------------------------
  # Accumulate the target and preds after each batch so that they
  # can be used for end-of-epoch calculations
  # ----------------------------   
  @classmethod
  def _accumulate_batch(cls, ctx):
    if (not cls.accumulate or (ctx.i_batch == cls.last_accumulated_batch)):
      # At the end of a batch, this method gets called multiple times, once for 
      # each child instance. However the accumulation needs to be done just once
      # per batch, as it is a singleton. So we do nothing if that batch data has
      # already been accumulated 
      return

    if (cls.yhat is None):
      # Initialise for the first batch of the epoch. Cloning copies the type, device
      # and size of the tensor.
      MetricsBaseCB.yhat = ctx.yhat.clone().detach()
      MetricsBaseCB.yb = ctx.yb.clone().detach()
    else:
      # Concatenate this batch data to the accumulated data from previous batches
      MetricsBaseCB.yhat = torch.cat((cls.yhat, ctx.yhat.detach()))
      MetricsBaseCB.yb = torch.cat((cls.yb, ctx.yb.detach()))

    # Mark that data for this batch has been accumulated
    MetricsBaseCB.last_accumulated_batch = ctx.i_batch

  # ----------------------------
  # Do any required calculations at the end of the epoch
  # ----------------------------   
  def _calc_epoch(self, ctx):
    pass

  # ----------------------------
  # Reset all metrics when training starts
  # ----------------------------   
  def begin_fit(self, ctx):
    self._reset(begin_fit=True)

  # ----------------------------
  # Reset all metrics at the beginning of each epoch
  # ----------------------------   
  def begin_epoch(self, ctx):
    self.value = 0. if (self.num_value == 1) else [0.] * self.num_value
    self._reset_accumulate()
    self._reset()

  # ----------------------------
  # Calculate training metrics for each batch
  # ----------------------------   
  def after_tr_loss(self, ctx):
    with torch.no_grad():
      self._calc_batch(ctx, is_val=False)

  # ----------------------------
  # Calculate validation metrics for each batch
  # ----------------------------   
  def after_val_loss(self, ctx):
    self._calc_batch(ctx)
    self._accumulate_batch(ctx)

  # ----------------------------
  # Print results at the end of each epoch
  # ----------------------------   
  def end_epoch(self, ctx):
    with torch.no_grad():
      self._calc_epoch(ctx)

#----------------------------------------------------
# Calculate an Average value per epoch using the given metric function
#----------------------------------------------------
class AverageMetricsCB(MetricsBaseCB):
  header = ['avg']

  def __init__(self, metric_fn):
    self.metric_fn = metric_fn
    self.header = [metric_fn.__name__]
    super().__init__()

  # ----------------------------
  # Count of total number of sample rows in an epoch
  # ----------------------------
  def _reset(self, begin_fit=False):
    self.total_sz = 0

  # ----------------------------
  # Add up the value returned by the metric function for each batch. It will be
  # averaged at the end of the epoch. Metrics are computed only for validation batches.
  # ----------------------------   
  def _calc_batch(self, ctx, is_val=True):
    if (is_val):
      batch_sz = ctx.yb.size(0)
      self.total_sz += batch_sz
      self.value += self.metric_fn(ctx.yhat, ctx.yb) * batch_sz

  # ----------------------------
  # Compute the average at the end of the epoch
  # ----------------------------
  def _calc_epoch(self, ctx):
    self.value = self.value / self.total_sz

#----------------------------------------------------
# Track the Training and Validation Loss per epoch, already 
# calculated by the loss function.
# This is the only metric which is computed for both training and validation
#----------------------------------------------------
class LossMetricsCB(MetricsBaseCB):
  header = ['tr_loss', 'smooth_loss', 'val_loss']
  beta=0.98

  # ----------------------------
  # Count of total number of sample rows in an epoch
  # ----------------------------
  def _reset(self, begin_fit=False):
    self.total_sz = 0

    # Initialise only once when training starts, but don't reset after each epoch
    if (begin_fit):
      self.mov_avg = 0
      self.n = 0

  # ----------------------------
  # Add up the loss for each training and validation batch
  # ----------------------------   
  def _calc_batch(self, ctx, is_val=True):
    if (is_val):
      # Validation loss
      batch_sz = ctx.yb.size(0)
      self.total_sz += batch_sz
      self.value[2] += ctx.loss * batch_sz
    else:
      # Training loss
      loss = ctx.loss
      self.value[0] += ctx.loss

      # Smoothed training loss based on moving average
      self.n += 1
      self.mov_avg = self.beta * self.mov_avg + (1 - self.beta) * loss
      self.value[1] = self.mov_avg / (1 - self.beta ** self.n)

  # ----------------------------
  # Compute the average at the end of the epoch
  # ----------------------------
  def _calc_epoch(self, ctx):
    # Calculate Training loss
    # Smoothed Training loss is already updated after each batch
    self.value[0] = self.value[0] / len(ctx.tr_dl)

    # Calculate Validation loss
    self.value[2] = self.value[2] / self.total_sz

#----------------------------------------------------
# Roc Auc score per epoch
#----------------------------------------------------
from sklearn.metrics import roc_auc_score
class RocAucMetricsCB(MetricsBaseCB):
  header = ['roc_auc']

  def _calc_epoch(self, ctx):
    yhat = torch.sigmoid(self.yhat)
    #yhat = F.softmax(ctx.yhat, dim=1)[:,-1]
    roc_score = roc_auc_score(self.yb.cpu().numpy(), yhat.cpu().numpy())
    self.value = roc_score


In [ ]:

#----------------------------------------------------
# Debug Callback to print yhat and loss after each batch
#----------------------------------------------------
class DebugYhatLossCB(Callback):
  def __init__(self, fwd=True, bkwd=False, prefix='\t'):
    self.fwd, self.bkwd = fwd, bkwd
    self.bkwd_pre = False # Don't show gradients values before backward()
    self.prefix = prefix

  def begin_tr(self, ctx):
    if (self.fwd):
      # Print Headers
      print (f'{self.prefix}Total {ctx.num_epochs} epochs with {ctx.num_batches} batches')
      print (f'{self.prefix}Batch, Pred, Loss')

  def after_tr_loss(self, ctx):
    dtr = ctx.dtr
    dtr.update_batch({'pred': ctx.yhat.mean().item(), 'loss': ctx.loss.item()})

    if (self.fwd):
      print (f'{self.prefix}{ctx.i_epoch:2}.{ctx.i_batch}, {ctx.yhat.mean():.4f}, {ctx.loss:.4f}')

    #if (self.bkwd_pre):
      #self.print_grad (ctx.model, before='Pre Gradients')

  def after_tr_backward(self, ctx):
    if (self.bkwd):
      dtr = ctx.dtr
      for name, param in ctx.model.named_parameters():
        dtr.add_param_layer('bkwd', name, param)

      #self.print_grad (ctx.model)

  def print_grad(self, model, before='\t\t'):
    hdr_name = 'Name'
    print (f'{before}{hdr_name:12}: Shape, Requires, Leaf, Grad')
    for name, param in model.named_parameters():
      print (f'{before}{name:12}: {tuple(param.data.size())}, {param.requires_grad}, {param.is_leaf}, {param.grad.mean() if param.grad is not None else 0:0.4G}')

In [ ]:

    batch_df = df[df['row_type'] == 'batch'][['run_id', 'batch_id', 'iter', 'batch_type', 'pred', 'loss']]
    if ('step_type' in df.columns):
      col_list = ['run_id', 'batch_id', 'iter', 'batch_type', 'step_operation', 'layer_name']
      if ('param_mean' in df.columns):
        col_list += ['param_mean', 'grad_mean']
      if ('out_activation' in df.columns):
        col_list += ['out_activation']
      layer_df = df[df['row_type'] == 'layer'][col_list]


  def add_layer(self, layer_data):
    layers = self.data['layers']
    layers.append(layer_data)

  #----------------------------------------------------
  # The last added step in the list is the current step
  #----------------------------------------------------
  def _get_run(self, run_idx):
    return self.runs[run_idx]


  def OLD_add_param_layer(self, operation_type, param_name, param, **kwargs):
    if (self.do and self._track_layer(param_name)):
      layer = dict()
      step_type = 'param'
      layer['step_type'] = step_type
      layer['operation_type'] = operation_type
      layer['data_item'] = param_name
      layer['param_mean'] = param.data.mean().item()
      layer['param_std'] = param.data.std().item()
      layer['requires_grad'] = param.requires_grad
      if (param.grad is not None):
        layer['grad_mean'] = param.grad.mean().item()
        layer['grad_std'] = param.data.std().item()
      if (kwargs is not None):
        layer.update(kwargs)

      current_batch = self._get_current_batch()
      current_iter = current_batch.data['iter'] # !!!! TODO don't access private data inside batch

      step_id = self._get_step_id(step_type, current_iter, operation_type)
      layer['step_id'] = step_id

      current_batch.add_layer(layer)

      self._create_layer(step_type, param_name, param, **kwargs)
 
      if (self.tb is not None):
        data_name, grad_name = param_name + '/data', param_name + '/grad'
        self.tb.add_hist(data_name, param.data, step_id)
        if (param.grad is not None):
          self.tb.add_hist(grad_name, param.grad, step_id)

  def OLD_add_activation_layer(self, operation_type, data_layer, out_activation, **kwargs):
    if (self.do and self._track_layer(data_layer)):
      layer = dict()
      step_type = 'activation'
      layer['step_type'] = step_type
      layer['operation_type'] = operation_type
      layer['data_item'] = data_layer
      layer['out_activation'] = out_activation.mean().item()

      if (kwargs is not None):
        layer.update(kwargs)

      current_batch = self._get_current_batch()
      current_iter = current_batch.data['iter'] # !!!! TODO don't access private data inside batch

      step_id = self._get_step_id(step_type, current_iter, operation_type)
      layer['step_id'] = step_id

      current_batch.add_layer(layer)

      self._create_layer(step_type, data_layer, out_activation, **kwargs)

      if (self.tb is not None):
        data_name = data_layer + '/activation'
        self.tb.add_hist(data_name, out_activation, step_id)


  def OLD_get_step_id(self, step_type, current_iter, current_operation):
    if ((current_operation != self.last_operation_type['operation_type']) or
        (current_iter != self.last_operation_type['iter'])):
      self.last_operation_type['operation_type'] = current_operation
      self.last_operation_type['iter'] = current_iter
      self.step_id[step_type] += 1

      step_id = self.step_id[step_type]
      self._create_step(step_id, current_operation, step_type)

    return self.step_id[step_type]

  def _create_run(self, num_epochs):
    self.OLD_last_operation_type = {'iter': -1, 'operation_type': ''}

    return new_run


**To view Tensorboard output locally, use ngrok to tunnel traffic to localhost. First, download and unzip ngrok on the Colab server**

In [ ]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

**Get TensorBoard running in the background**

In [ ]:
# Set the LOGDIR correctly to use Tensorboard
LOG_DIR = 'tbtry'

In [ ]:
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

**Launch ngrok background process**

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

**We get the public URL where we can access the colab TensorBoard web page. This will output a URL you can click on**

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://978255fa.ngrok.io
